## Теоретическая часть

#### 1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? 


Колларобативная фильтрация типа User-User рекомендации отличаются от Item-Item рекомендаций тем, что они создаются на разной основе. Первые означают, что мы выибраем подобных пользователей и на этой основе делаем рекомендацию по товарам, а вторые создаются на основе поиска аналогично покупаемых товаров, а рекомендация дается уже пользователям.  

Одинаковые же они в том, что и там и там рекомендация создается на основе подбора аналога (либо товара, либо покупателя) выбора аналогичных товаров. То есть у обоих подходов общий алгоритм, но разные субьекты, принимающие в нем участие.

Кроме того, Item-user матрица является исходной матрицей как для user-based, так и для item-based коллаборативной фильтрации

У user-based подхода есть существенный недостаток: с увеличением количества пользователей линейно увеличивается сложность вычисления всей персонализированной рекомендации.

Когда количество объектов для рекомендаций большое, затраты на user-based подход могут быть оправданы. Однако во многих сервисах (например в ivi.ru), количество объектов в разы меньше количества пользователей. Для таких случаев и был придуман и чаще используется именно item-based подход, уменьшающий сложность вычислений.

#### Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  b

Как я понял - можно, но нужно будет использовать не транспорированную матрицу взаимодествия item_to_item, а прямую user_to_user.

#### В чем принципиальные отличия item-item рекомендаций от ALS?

- В ALS есть интерации, позволяющие разложить матрицу предпочтений на произведение 2-х матриц (факторов пользователей U и факторов товаров I), работая по принципу минимизации среднеквадратичной ошибки на проставленых рейтингах. 

- В ALS оптимизация происходит поочередно по факторам пользователей, потом по факторам товаров.

- В ALS для обхода переобучения к среднеквадратинчной ошибке добавляются коэффициенты регуляризация 

- Также в ALS для улучшения качества результата применяются различные алгоритмы разложения матриц: TF-IDF, BM-25 и т.д.

#### 2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


- Trust-based recommendations, в которых учитывается не только близость людей по интересам, но также их «социальная» близость и степень доверия между ними. Подобный вес "доверия" премирует результат, полученный от соседки, с которой есть иные коллаборации. Напримерб Если например видим, что на фейсбуке девушка периодически заходит на страницу с аудиозаписями подруги, значит доверяет её музыкальному вкусу. Следовательно в рекомендации девушке можно вполне подмешивать новые песни из плейлиста подруги.

- кол-во лайков/дизлайков как штраф/премия при регуляризации модели

#### 3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


- Большое количество товаров, что ведет к увеличению размерности матрицы и большему ресурсу, необходимому для её обработки

- Проблема холодного старта, т.е. когда нет оснований (данных) для принятия первоначальных решений

- Отсутствие единого алгоритма, т.к. каждый раз приходится приспосабливаться под тип и кол-во данных, а также факторы на них влияющие.

## Практическая часть


In [3]:
!pip install implicit

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# !pip install google.colab

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
# root = '/content/drive/My Drive/GB/RecSys/webinar_2/'

In [5]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

C:\Users\alexa\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [ ]:
for factor in [50, 100, 300]:
    model = ALS(factor, ...)

NameError: ignored

**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!

In [ ]:
!pip install hyperopt

# Use hyperopt!